In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import json
import os
import sklearn as sk

In [64]:
classes_train = pd.read_csv('../data/train/_classes.csv')
classes_test = pd.read_csv('../data/test/_classes.csv')
classes_valid = pd.read_csv('../data/valid/_classes.csv')

# Concatenate all data
all_data_merged = pd.concat([classes_train, classes_test, classes_valid])

melted = all_data_merged.melt(id_vars=["filename"], var_name="category", value_name="value")
melted = melted[melted["value"] == 1]
all_data_merged_class = melted.drop("value", axis=1).rename(columns={"category": "class"})

all_data_merged_class.head()

subdatasets = [[] for _ in range(5)]

# Create a dictionary to map filenames to their full rows
filename_to_row = {row['filename']: row for _, row in all_data_merged.iterrows()}

# Distribute entries into the subdatasets
for idx, row in all_data_merged_class.iterrows():
    full_row = filename_to_row[row['filename']]
    subdatasets[idx % 5].append(full_row)

# Convert lists to DataFrames
subdatasets = [pd.DataFrame(subdataset) for subdataset in subdatasets]

# Verify the first few rows of each subdataset
for i, subdataset in enumerate(subdatasets):
    subdataset.sample(frac=1).reset_index(drop=True)

#shuffle the subdatasets
for i, subdataset in enumerate(subdatasets):
    subdatasets[i] = subdataset.sample(frac=1).reset_index(drop=True)
    



In [67]:
#write subdatasets to csv files in data/cross_valid_sets
for i, subdataset in enumerate(subdatasets):
    subdataset.to_csv(f'../data/cross_valid_sets/subdataset_{i}.csv', index=False)
    